In [17]:
import torch
import sys
sys.path.append("/DRIVEcodeV1/DCG_Core")
sys.path.append("/your/dataset/path")#default ckpts files saving path 
import pytorch_lightning as pl
from module_copy1 import *
from model_copy1 import *
from model_copy2 import *
from main_copy1 import save_preds
from tqdm import tqdm
import os


In [18]:
teachmodel=TVTN(multitask=task, backbone="none", concept_features=True, device = f"cuda:0", train_concepts=False)
model = VTN(multitask=task, backbone= "none", concept_features=True, device = f"cuda:0", train_concepts=False)

using concept features
CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768

In [19]:
def save_preds(logits, target, save_name, p):
    b, s = target.shape
    df = pd.DataFrame()
    df['logits'] = logits.squeeze().reshape(b*s).tolist()
    df['target'] = target.squeeze().reshape(b*s).tolist()

    directory = os.path.dirname(f'{p}/{save_name}.csv')

    os.makedirs(directory, exist_ok=True)
    
    df.to_csv(f'{p}/{save_name}.csv', mode='a', index=False, header=False)

In [20]:
def get_ckpt_files(ckpts_path):
    ckpt_files = []
    for filename in os.listdir(ckpts_path):

        if filename.endswith('.ckpt'):
            full_path = os.path.join(ckpts_path, filename)
            ckpt_files.append(full_path)
    return ckpt_files

In [21]:
img_noise=None
task = "multitask"#angle/distance/multitask ,align with version path
version_path="/hpc2hdd/home/tianlangxue/XAI4AD/comma2k19data/ckpts_final/ckpts_final_comma_multitask_none_True_1/lightning_logs/version_52_5loss"
prediction_save_path ="/hpc2hdd/home/tianlangxue/XAI4AD/concept_gridlock/New/predictions"
text_save_rootpath =f"/your/results_path/MAEtext/{task}_{os.path.basename(version_path)}.txt"
ckpts_path =version_path+"/checkpoints"
ckpt_path_list = get_ckpt_files(ckpts_path)

In [22]:
import pandas as pd
def calculate_mae(actual, predicted):
    return ((actual - predicted).abs()).mean()

def read_csv_file(file_path):
    return pd.read_csv(file_path)

def write_to_txt_file(txt_file_path, file_name, mae_value):
    try:
        with open(txt_file_path, 'a') as file:
            file.write(f"{file_name},{mae_value}\n")
    except Exception as e:
        print(f"Failed to write to {txt_file_path}: {e}")
        
def cal_save_MAE(csvpathlist,textpath,task):
    thisbranch_MAE=[]
    txt_file_path = textpath+f"{task}_{os.path.basename(version_path)}_clean.txt" 
    for csv_file_path in csvpathlist:  

        df = read_csv_file(csv_file_path)

        if df.shape[1] < 2:
            raise ValueError("CSV 文件需要至少有两列数据。")
        actual_values = df.iloc[:, 1]
        predicted_values = df.iloc[:, 0]

        mae = calculate_mae(actual_values, predicted_values)
        if task == "multitask":
            if "angle" in csv_file_path:
                txt_file_path = f"/your/results_path/MAEtext/angle_{task}_{os.path.basename(version_path)}_ptbconc.txt"
            elif "dist" in csv_file_path:
                txt_file_path = f"/your/results_path/MAEtext/distance_{task}_{os.path.basename(version_path)}_ptbconc.txt"
        file_name = csv_file_path.split('/')[-1]

        write_to_txt_file(txt_file_path, file_name, mae)
        thisbranch_MAE.append(mae)
    return thisbranch_MAE

In [23]:
MAE_list=[]#save num of epochs mae results

prediction_save_path ="/yourpath/DCGPreds"
text_save_rootpath="/yourpath/DCGMAE"
from dataloader_comma import *
if not os.path.exists(prediction_save_path):
    os.makedirs(prediction_save_path)

for id in tqdm(range(len(ckpt_path_list))):
    
    print(os.path.basename(ckpt_path_list[id]).split("-")[0])
    testmodule = LaneModule.load_from_checkpoint(
    ckpt_path_list[id],
    model=model,  
    teachmodel =teachmodel,
    multitask=task,
    dataset = "comma",
    dataset_path="/your/dataset_path",
    bs=1,  
    ground_truth="desired",
    img_noise=img_noise,
    # dataset_path=args.dataset_path,
    # dataset_fraction=args.dataset_fraction
)

    trainer = pl.Trainer(
        gpus=1,
        accelerator='gpu',
        devices="cuda:0",
    )

    preds = trainer.predict(testmodule)

    csvp=set()
    print(len(preds))
    assert len(preds) >= 3, "pred must have at least three elements"
    for pred in preds:
        if task != "multitask":
            predictions, preds_1, preds_2 = pred[0], pred[1], pred[2] 
            save_preds(predictions, preds_1, f"single_{task}_DRIVE_100scenarios_{img_noise}", prediction_save_path)#DRIVE=concept
            csvp.add(prediction_save_path+f"/single_{task}_DRIVE_100scenarios_{img_noise}.csv")
        else:
            preds, angle, dist = pred[0], pred[1], pred[2]
            preds_angle, preds_dist = preds[0], preds[1]
            # print(preds_angle.shape,preds_dist.shape)
            # print(angle.shape,dist.shape)
            save_preds(preds_angle, angle, f"angle_multi_{task}_DRIVE_100scenarios_{img_noise}", prediction_save_path)
            save_preds(preds_dist, dist, f"dist_multi_{task}_DRIVE_100scenarios_{img_noise}", prediction_save_path)
            csvp.add(prediction_save_path+f"/angle_multi_{task}_DRIVE_100scenarios_{img_noise}.csv")
            csvp.add((prediction_save_path+f"/dist_multi_{task}_DRIVE_100scenarios_{img_noise}.csv"))
    csvp =list(csvp)
    print(csvp)
    MAE_list.append(cal_save_MAE(csvp,text_save_rootpath,task))

  0%|          | 0/40 [00:00<?, ?it/s]

epoch=39


/hpc2hdd/home/tianlangxue/anaconda3/envs/OPT/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/hpc2hdd/home/tianlangxue/anaconda3/envs/OPT/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:498: UserWarning: The flag `devices=cuda:0` will be ignored, instead the device specific number 1 will be used
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.o

data_path----- /hpc2hdd/home/tianlangxue/XAI4AD/comma2k19data/comma_test_w_desired_filtered.h5py -----------------
Predicting DataLoader 0: 100%|██████████| 14/14 [00:04<00:00,  2.89it/s]
14


  0%|          | 0/40 [00:14<?, ?it/s]


RuntimeError: shape '[480]' is invalid for input of size 240